In [3]:
def convert_data(input_data: str):
    """
    Converts tab-separated value-error pairs into the format:
    (index,value) +- (0,error)
    
    Args:
        input_data (str): Multiline string with each line containing a value and error separated by a tab.
    
    Returns:
        list of str: Formatted strings.
    """
    lines = input_data.strip().split('\n')
    print(lines)
    formatted_output = []

    for i, line in enumerate(lines, start=1):
        value, error = line.split()
        formatted_output.append(f"({i},{value}) +- (0,{error})")

    print('\n'.join(formatted_output))

In [4]:
convert_data("""8.7	0.97872097
18.35	3.483419373
13.8	1.472555959
17.5	2.039091645
26.8	3.427443423
30.55	5.093597633
31.7	3.686390333
40.05	3.619755735
33.9	3.17722156
41.4	6.628487166
22.75	2.633288918
23.1	4.228972751
29.2	3.819616955
34.3	4.791439735
54.25	5.998903409
""")

['8.7\t0.97872097', '18.35\t3.483419373', '13.8\t1.472555959', '17.5\t2.039091645', '26.8\t3.427443423', '30.55\t5.093597633', '31.7\t3.686390333', '40.05\t3.619755735', '33.9\t3.17722156', '41.4\t6.628487166', '22.75\t2.633288918', '23.1\t4.228972751', '29.2\t3.819616955', '34.3\t4.791439735', '54.25\t5.998903409']
(1,8.7) +- (0,0.97872097)
(2,18.35) +- (0,3.483419373)
(3,13.8) +- (0,1.472555959)
(4,17.5) +- (0,2.039091645)
(5,26.8) +- (0,3.427443423)
(6,30.55) +- (0,5.093597633)
(7,31.7) +- (0,3.686390333)
(8,40.05) +- (0,3.619755735)
(9,33.9) +- (0,3.17722156)
(10,41.4) +- (0,6.628487166)
(11,22.75) +- (0,2.633288918)
(12,23.1) +- (0,4.228972751)
(13,29.2) +- (0,3.819616955)
(14,34.3) +- (0,4.791439735)
(15,54.25) +- (0,5.998903409)


In [1]:
from unified_planning.shortcuts import *
from unified_planning.model.phgn import *
from unified_planning.model.phgn.goal_network import PartialOrderGoalNetwork
from phgn_planner.config import UCTConfig
from unified_planning.engines import PHGNSimulator

In [2]:
problem = PHGNProblem()

Location = UserType("Location")
GroundLocation = UserType("GroundLocation", father=Location)
RobotLocation = UserType("RobotLocation", father=Location)
Locatable = UserType("Locatable")
Robot = UserType("Robot", father=Locatable)
Key = UserType("Key", father=Locatable)
Box = UserType("Box", father=Locatable)

loc = problem.add_fluent("loc", Location, obj=Locatable)
adjacent = problem.add_fluent("adjacent", BoolType(), default_initial_value=False, x=GroundLocation, y=GroundLocation)
locked = problem.add_fluent("locked", BoolType(), default_initial_value=False, x=GroundLocation, y=GroundLocation)

l1 = problem.add_object("l1", GroundLocation)
l2 = problem.add_object("l2", GroundLocation)
l3 = problem.add_object("l3", GroundLocation)
l4 = problem.add_object("l4", GroundLocation)
l5 = problem.add_object("l5", GroundLocation)
l6 = problem.add_object("l6", GroundLocation)
# l7 = problem.add_object("l7", GroundLocation)
# l8 = problem.add_object("l8", GroundLocation)
# l9 = problem.add_object("l9", GroundLocation)
on_robot = problem.add_object("on_robot", RobotLocation)
robot = problem.add_object("robot", Robot)
key = problem.add_object("key", Key)
box = problem.add_object("box", Box)

# for x, y in [(l1, l2), (l1, l4), (l2, l1), (l2, l5), (l2, l3), (l3, l2), (l3, l6), (l4, l1), (l4, l7), (l5, l2), (l6, l3), (l6, l9), (l7, l4), (l7, l8), (l8, l7), (l8, l9), (l9, l6), (l9, l8)]:
#     problem.set_initial_value(adjacent(x, y), True)
# for x, y in [(l4, l5), (l5, l4), (l5, l6), (l6, l5), (l5, l8), (l8, l5), (l1, l2), (l1, l4), (l2, l1), (l2, l5), (l2, l3), (l3, l2), (l3, l6), (l4, l1), (l4, l7), (l5, l2), (l6, l3), (l6, l9), (l7, l4), (l7, l8), (l8, l7), (l8, l9), (l9, l6), (l9, l8)]:
#     problem.set_initial_value(adjacent(x, y), True)
for x, y in [(l1, l2), (l1, l4), (l2, l1), (l2, l5), (l2, l3), (l3, l2), (l3, l6), (l4, l1), (l5, l2), (l6, l3)]:
    problem.set_initial_value(adjacent(x, y), True)
problem.set_initial_value(locked(l2, l5), True)
# problem.set_initial_value(locked(l4, l5), True)
# problem.set_initial_value(locked(l6, l5), True)
# problem.set_initial_value(locked(l8, l5), True)
problem.set_initial_value(loc(robot), l3)
problem.set_initial_value(loc(key), l6)
problem.set_initial_value(loc(box), l5)

# Declare Goals
problem.add_goal(loc(box).Equals(l4))

# Declare Actions
move_without_key = ProbabilisticAction("move_without_key", from_loc=GroundLocation, to_loc=GroundLocation)
move_without_key.add_precondition(adjacent(move_without_key.from_loc, move_without_key.to_loc))
move_without_key.add_precondition(locked(move_without_key.from_loc, move_without_key.to_loc).Not())
move_without_key.add_precondition(loc(robot).Equals(move_without_key.from_loc))
move_without_key.add_precondition(loc(key).Equals(on_robot).Not())
move_without_key.add_outcome("success", 0.9)
move_without_key.add_effect("success", loc(robot), move_without_key.to_loc)
move_without_key.add_outcome("failure", 0.1)
problem.add_action(move_without_key)

move_with_key = ProbabilisticAction("move_with_key", from_loc=GroundLocation, to_loc=GroundLocation)
move_with_key.add_precondition(adjacent(move_with_key.from_loc, move_with_key.to_loc))
move_with_key.add_precondition(loc(key).Equals(on_robot))
move_with_key.add_precondition(loc(robot).Equals(move_with_key.from_loc))
move_with_key.add_outcome("success", 0.8)
move_with_key.add_effect("success", loc(robot), move_with_key.to_loc)
move_with_key.add_outcome("drop_key", 0.1)
move_with_key.add_effect("drop_key", loc(robot), move_with_key.to_loc)
move_with_key.add_effect("drop_key", loc(key), move_with_key.to_loc)
move_with_key.add_outcome("failure", 0.1)
problem.add_action(move_with_key)

pickup_key = ProbabilisticAction("pickup_key", key_loc=GroundLocation)
pickup_key.add_precondition(loc(key).Equals(pickup_key.key_loc))
pickup_key.add_precondition(loc(robot).Equals(pickup_key.key_loc))
pickup_key.add_outcome("success", 0.9)
pickup_key.add_effect("success", loc(key), on_robot)
pickup_key.add_outcome("failure", 0.1)
problem.add_action(pickup_key)

pickup_box = ProbabilisticAction("pickup_box", box_loc=GroundLocation)
pickup_box.add_precondition(loc(box).Equals(pickup_box.box_loc))
pickup_box.add_precondition(loc(robot).Equals(pickup_box.box_loc))
pickup_box.add_outcome("success", 0.9)
pickup_box.add_effect("success", loc(box), on_robot)
pickup_box.add_outcome("failure", 0.1)
problem.add_action(pickup_box)

putdown_key = ProbabilisticAction("putdown_key", key_loc=GroundLocation)
putdown_key.add_precondition(loc(key).Equals(on_robot))
putdown_key.add_precondition(loc(robot).Equals(putdown_key.key_loc))
putdown_key.add_outcome("success", 1)
putdown_key.add_effect("success", loc(key), putdown_key.key_loc)
problem.add_action(putdown_key)

putdown_box = ProbabilisticAction("putdown_box", box_loc=GroundLocation)
putdown_box.add_precondition(loc(box).Equals(on_robot))
putdown_box.add_precondition(loc(robot).Equals(putdown_box.box_loc))
putdown_box.add_outcome("success", 1)
putdown_box.add_effect("success", loc(box), putdown_box.box_loc)
problem.add_action(putdown_box)

move_box = PHGNMethod("move_box", to_loc=GroundLocation)
move_box.add_precondition(loc(box).Equals(on_robot).Not())
gn = PartialOrderGoalNetwork()
gn.add(loc(box).Equals(on_robot), And(loc(box).Equals(on_robot), loc(robot).Equals(move_box.to_loc)), loc(box).Equals(move_box.to_loc))
move_box.set_goal_network(gn)
problem.add_method(move_box)

get_box = PHGNMethod("get_box", box_loc=GroundLocation)
get_box.add_precondition(loc(box).Equals(get_box.box_loc))
get_box.add_precondition(loc(robot).Equals(get_box.box_loc).Not())
get_box.add_precondition(loc(key).Equals(on_robot).Not())
gn = PartialOrderGoalNetwork()
gn.add(loc(key).Equals(on_robot), And(loc(robot).Equals(get_box.box_loc), loc(key).Equals(on_robot)), loc(box).Equals(on_robot))
get_box.set_goal_network(gn)
problem.add_method(get_box)

get_key = PHGNMethod("get_key", key_loc=GroundLocation)
get_key.add_precondition(loc(key).Equals(get_key.key_loc))
get_key.add_precondition(loc(robot).Equals(get_key.key_loc).Not())
gn = PartialOrderGoalNetwork()
gn.add(loc(robot).Equals(get_key.key_loc), loc(key).Equals(on_robot))
get_key.set_goal_network(gn)
problem.add_method(get_key)

gn = PartialOrderGoalNetwork()
gn.add(loc(box).Equals(l4))
problem.set_goal_network(gn)

In [3]:
# Initialize the problem
problem = PHGNProblem()

# 1. Define UserTypes (with hierarchy)

Object = UserType("Object")

Place = UserType("Place", father=Object)
Locatable = UserType("Locatable", father=Object)

Depot = UserType("Depot", father=Place)
Distributor = UserType("Distributor", father=Place)

Truck = UserType("Truck", father=Locatable)
Hoist = UserType("Hoist", father=Locatable)
Surface = UserType("Surface", father=Locatable)

Pallet = UserType("Pallet", father=Surface)
Crate = UserType("Crate", father=Surface)

# 2. Define Fluents (Predicates)

# (at ?x - locatable ?y - place)
at = problem.add_fluent(
    "at", BoolType(), default_initial_value=False, x=Locatable, y=Place
)

# (on ?x - crate ?y - surface)
on = problem.add_fluent(
    "on", BoolType(), default_initial_value=False, x=Crate, y=Surface
)

# (in ?x - crate ?y - truck)
in_truck = problem.add_fluent(
    "in_truck", BoolType(), default_initial_value=False, x=Crate, y=Truck
)

# (lifting ?x - hoist ?y - crate)
lifting = problem.add_fluent(
    "lifting", BoolType(), default_initial_value=False, x=Hoist, y=Crate
)

# (available ?x - hoist)
available = problem.add_fluent(
    "available", BoolType(), default_initial_value=False, x=Hoist
)

# (clear ?x - surface)
clear = problem.add_fluent(
    "clear", BoolType(), default_initial_value=False, x=Surface
)

# (supervisor_needed ?x - hoist ?y - crate ?z - truck ?p - place)
supervisor_needed = problem.add_fluent(
    "supervisor_needed",
    BoolType(),
    default_initial_value=False,
    x=Hoist,
    y=Crate,
    z=Truck,
    p=Place,
)

# 3. Define Actions

# (:action Drive
#   :parameters (?x - truck ?y - place ?z - place)
#   :precondition (and (at ?x ?y))
#   :effect (and (not (at ?x ?y)) (at ?x ?z))
# )
drive = InstantaneousAction("drive", x=Truck, y=Place, z=Place)
drive.add_precondition(at(drive.x, drive.y))
drive.add_effect(at(drive.x, drive.y), False)
drive.add_effect(at(drive.x, drive.z), True)
problem.add_action(drive)

# (:action Lift
#   :parameters (?x - hoist ?y - crate ?z - surface ?p - place)
#   :precondition (and
#     (at ?x ?p) (available ?x) (at ?y ?p) (on ?y ?z) (clear ?y)
#   )
#   :effect (and
#     (not (at ?y ?p)) (lifting ?x ?y) (not (clear ?y))
#     (not (available ?x)) (clear ?z) (not (on ?y ?z))
#   )
# )
lift = InstantaneousAction("lift", x=Hoist, y=Crate, z=Surface, p=Place)
lift.add_precondition(at(lift.x, lift.p))
lift.add_precondition(available(lift.x))
lift.add_precondition(at(lift.y, lift.p))
lift.add_precondition(on(lift.y, lift.z))
lift.add_precondition(clear(lift.y))
lift.add_effect(at(lift.y, lift.p), False)
lift.add_effect(lifting(lift.x, lift.y), True)
lift.add_effect(clear(lift.y), False)
lift.add_effect(available(lift.x), False)
lift.add_effect(clear(lift.z), True)
lift.add_effect(on(lift.y, lift.z), False)
problem.add_action(lift)

# (:action Drop
#   :parameters (?x - hoist ?y - crate ?z - surface ?p - place)
#   :precondition (and
#     (at ?x ?p) (at ?z ?p) (clear ?z) (lifting ?x ?y)
#   )
#   :effect (and
#     (available ?x) (not (lifting ?x ?y)) (at ?y ?p)
#     (not (clear ?z)) (clear ?y) (on ?y ?z)
#   )
# )
drop = InstantaneousAction("drop", x=Hoist, y=Crate, z=Surface, p=Place)
drop.add_precondition(at(drop.x, drop.p))
drop.add_precondition(at(drop.z, drop.p))
drop.add_precondition(clear(drop.z))
drop.add_precondition(lifting(drop.x, drop.y))
drop.add_effect(available(drop.x), True)
drop.add_effect(lifting(drop.x, drop.y), False)
drop.add_effect(at(drop.y, drop.p), True)
drop.add_effect(clear(drop.z), False)
drop.add_effect(clear(drop.y), True)
drop.add_effect(on(drop.y, drop.z), True)
problem.add_action(drop)

# (:action Unsupervised_Load
#   :parameters (?x - hoist ?y - crate ?z - truck ?p - place)
#   :precondition (and
#     (at ?x ?p) (at ?z ?p) (lifting ?x ?y)
#   )
#   :effect ( probabilistic
#     0.5 (and (not (lifting ?x ?y)) (in ?y ?z) (available ?x))
#     0.5 (and (supervisor_needed ?x ?y ?z ?p))
#   )
# )
unsupervised_load = ProbabilisticAction(
    "unsupervised_load", x=Hoist, y=Crate, z=Truck, p=Place
)
unsupervised_load.add_precondition(at(unsupervised_load.x, unsupervised_load.p))
unsupervised_load.add_precondition(at(unsupervised_load.z, unsupervised_load.p))
unsupervised_load.add_precondition(
    lifting(unsupervised_load.x, unsupervised_load.y)
)

# Outcome 1
unsupervised_load.add_outcome("success", 0.5)
unsupervised_load.add_effect(
    "success", lifting(unsupervised_load.x, unsupervised_load.y), False
)
unsupervised_load.add_effect(
    "success", in_truck(unsupervised_load.y, unsupervised_load.z), True
)
unsupervised_load.add_effect("success", available(unsupervised_load.x), True)

# Outcome 2
unsupervised_load.add_outcome("failure", 0.5)
unsupervised_load.add_effect(
    "failure",
    supervisor_needed(
        unsupervised_load.x,
        unsupervised_load.y,
        unsupervised_load.z,
        unsupervised_load.p,
    ),
    True,
)
problem.add_action(unsupervised_load)

# (:action Supervised_Load
#   :parameters (?x - hoist ?y - crate ?z - truck ?p - place)
#   :precondition (and
#     (at ?x ?p) (at ?z ?p) (lifting ?x ?y) (supervisor_needed ?x ?y ?z ?p)
#   )
#   :effect (and (not (lifting ?x ?y)) (in ?y ?z) (available ?x) (not (supervisor_needed ?x ?y ?z ?p)))
# )
supervised_load = InstantaneousAction(
    "supervised_load", x=Hoist, y=Crate, z=Truck, p=Place
)
supervised_load.add_precondition(at(supervised_load.x, supervised_load.p))
supervised_load.add_precondition(at(supervised_load.z, supervised_load.p))
supervised_load.add_precondition(lifting(supervised_load.x, supervised_load.y))
supervised_load.add_precondition(
    supervisor_needed(
        supervised_load.x, supervised_load.y, supervised_load.z, supervised_load.p
    )
)
supervised_load.add_effect(lifting(supervised_load.x, supervised_load.y), False)
supervised_load.add_effect(in_truck(supervised_load.y, supervised_load.z), True)
supervised_load.add_effect(available(supervised_load.x), True)
supervised_load.add_effect(
    supervisor_needed(
        supervised_load.x, supervised_load.y, supervised_load.z, supervised_load.p
    ),
    False,
)
problem.add_action(supervised_load)

# (:action Unsupervised_Unload
#   :parameters (?x - hoist ?y - crate ?z - truck ?p - place)
#   :precondition (and
#     (at ?x ?p) (at ?z ?p) (available ?x) (in ?y ?z)
#   )
#   :effect ( probabilistic
#     0.5 (and (not (in ?y ?z)) (not (available ?x)) (lifting ?x ?y))
#     0.5 (and (supervisor_needed ?x ?y ?z ?p))
#   )
# )
unsupervised_unload = ProbabilisticAction(
    "unsupervised_unload", x=Hoist, y=Crate, z=Truck, p=Place
)
unsupervised_unload.add_precondition(
    at(unsupervised_unload.x, unsupervised_unload.p)
)
unsupervised_unload.add_precondition(
    at(unsupervised_unload.z, unsupervised_unload.p)
)
unsupervised_unload.add_precondition(available(unsupervised_unload.x))
unsupervised_unload.add_precondition(
    in_truck(unsupervised_unload.y, unsupervised_unload.z)
)

# Outcome 1
unsupervised_unload.add_outcome("success", 0.5)
unsupervised_unload.add_effect(
    "success", in_truck(unsupervised_unload.y, unsupervised_unload.z), False
)
unsupervised_unload.add_effect("success", available(unsupervised_unload.x), False)
unsupervised_unload.add_effect(
    "success", lifting(unsupervised_unload.x, unsupervised_unload.y), True
)

# Outcome 2
unsupervised_unload.add_outcome("failure", 0.5)
unsupervised_unload.add_effect(
    "failure",
    supervisor_needed(
        unsupervised_unload.x,
        unsupervised_unload.y,
        unsupervised_unload.z,
        unsupervised_unload.p,
    ),
    True,
)
problem.add_action(unsupervised_unload)

# (:action Supervised_Unload
#   :parameters (?x - hoist ?y - crate ?z - truck ?p - place)
#   :precondition (and
#     (at ?x ?p) (at ?z ?p) (available ?x) (in ?y ?z) (supervisor_needed ?x ?y ?z ?p)
#   )
#   :effect (and (not (in ?y ?z)) (not (available ?x)) (lifting ?x ?y) (not (supervisor_needed ?x ?y ?z ?p)))
# )
supervised_unload = InstantaneousAction(
    "supervised_unload", x=Hoist, y=Crate, z=Truck, p=Place
)
supervised_unload.add_precondition(at(supervised_unload.x, supervised_unload.p))
supervised_unload.add_precondition(at(supervised_unload.z, supervised_unload.p))
supervised_unload.add_precondition(available(supervised_unload.x))
supervised_unload.add_precondition(
    in_truck(supervised_unload.y, supervised_unload.z)
)
supervised_unload.add_precondition(
    supervisor_needed(
        supervised_unload.x,
        supervised_unload.y,
        supervised_unload.z,
        supervised_unload.p,
    )
)
supervised_unload.add_effect(
    in_truck(supervised_unload.y, supervised_unload.z), False
)
supervised_unload.add_effect(available(supervised_unload.x), False)
supervised_unload.add_effect(
    lifting(supervised_unload.x, supervised_unload.y), True
)
supervised_unload.add_effect(
    supervisor_needed(
        supervised_unload.x,
        supervised_unload.y,
        supervised_unload.z,
        supervised_unload.p,
    ),
    False,
)
problem.add_action(supervised_unload)

put_on_1 = PHGNMethod(
    "put_on_1",
    c=Crate,
    s2=Surface,
    p=Place,
    h=Hoist,
)
put_on_1.add_precondition(at(put_on_1.c, put_on_1.p))
put_on_1.add_precondition(at(put_on_1.s2, put_on_1.p))
put_on_1.add_precondition(at(put_on_1.h, put_on_1.p))
gn = PartialOrderGoalNetwork()
gn.add(
    And(clear(put_on_1.c), clear(put_on_1.s2)),
    lifting(put_on_1.h, put_on_1.c),
    on(put_on_1.c, put_on_1.s2),
)
put_on_1.set_goal_network(gn)
problem.add_method(put_on_1)

put_on_2 = PHGNMethod(
    "put_on_2",
    c=Crate,
    s2=Surface,
    p=Place,
    t=Truck,
    h=Hoist,
)
put_on_2.add_precondition(in_truck(put_on_2.c, put_on_2.t))
gn = PartialOrderGoalNetwork()
gn.add(
    at(put_on_2.t, put_on_2.p),
    clear(put_on_2.s2),
    lifting(put_on_2.h, put_on_2.c),
    on(put_on_2.c, put_on_2.s2),
)
put_on_2.set_goal_network(gn)
problem.add_method(put_on_2)

put_on_3 = PHGNMethod(
    "put_on_3",
    c=Crate,
    s2=Surface,
    p1=Place,
    p2=Place,
    t=Truck,
)
put_on_3.add_precondition(at(put_on_3.c, put_on_3.p1))
put_on_3.add_precondition(at(put_on_3.s2, put_on_3.p2))
put_on_3.add_precondition(Not(Equals(put_on_3.p1, put_on_3.p2)))
gn = PartialOrderGoalNetwork()
gn.add(
    in_truck(put_on_3.c, put_on_3.t),
    And(in_truck(put_on_3.c, put_on_3.t), at(put_on_3.t, put_on_3.p2)),
    on(put_on_3.c, put_on_3.s2),
)
put_on_3.set_goal_network(gn)
problem.add_method(put_on_3)

do_clear = PHGNMethod(
    "do_clear",
    s1=Surface,
    c=Crate,
)
do_clear.add_precondition(Not(clear(do_clear.s1)))
do_clear.add_precondition(on(do_clear.c, do_clear.s1))
gn = PartialOrderGoalNetwork()
gn.add(
    clear(do_clear.c),
    clear(do_clear.s1),
)
do_clear.set_goal_network(gn)
problem.add_method(do_clear)

lift_crate = PHGNMethod(
    "lift_crate",
    c=Crate,
    p=Place,
    h=Hoist,
    t=Truck,
)
lift_crate.add_precondition(in_truck(lift_crate.c, lift_crate.t))
lift_crate.add_precondition(at(lift_crate.h, lift_crate.p))
gn = PartialOrderGoalNetwork()
gn.add(
    at(lift_crate.t, lift_crate.p),
    lifting(lift_crate.h, lift_crate.c),
)
lift_crate.set_goal_network(gn)
problem.add_method(lift_crate)

load_truck = PHGNMethod(
    "load_truck",
    c=Crate,
    s=Surface,
    p=Place,
    t=Truck,
    h=Hoist,
)
load_truck.add_precondition(at(load_truck.c, load_truck.p))
load_truck.add_precondition(at(load_truck.s, load_truck.p))
load_truck.add_precondition(on(load_truck.c, load_truck.s))
load_truck.add_precondition(at(load_truck.h, load_truck.p))
gn = PartialOrderGoalNetwork()
gn.add(
    at(load_truck.t, load_truck.p),
    clear(load_truck.c),
    lifting(load_truck.h, load_truck.c),
    in_truck(load_truck.c, load_truck.t),
)
load_truck.set_goal_network(gn)
problem.add_method(load_truck)

unload_truck = PHGNMethod(
    "unload_truck",
    c=Crate,
    s=Surface,
    p=Place,
    t=Truck,
    h=Hoist,
)
unload_truck.add_precondition(in_truck(unload_truck.c, unload_truck.t))
unload_truck.add_precondition(at(unload_truck.t, unload_truck.p))
unload_truck.add_precondition(at(unload_truck.h, unload_truck.p))
unload_truck.add_precondition(at(unload_truck.s, unload_truck.p))
gn = PartialOrderGoalNetwork()
gn.add(
    clear(unload_truck.s),
    lifting(unload_truck.h, unload_truck.c),
    on(unload_truck.c, unload_truck.s),
)
unload_truck.set_goal_network(gn)
problem.add_method(unload_truck)

# 1. Define Objects
depot0 = problem.add_object("depot0", Depot)
distributor0 = problem.add_object("distributor0", Distributor)
distributor1 = problem.add_object("distributor1", Distributor)
truck0 = problem.add_object("truck0", Truck)
truck1 = problem.add_object("truck1", Truck)
pallet0 = problem.add_object("pallet0", Pallet)
pallet1 = problem.add_object("pallet1", Pallet)
pallet2 = problem.add_object("pallet2", Pallet)
crate0 = problem.add_object("crate0", Crate)
crate1 = problem.add_object("crate1", Crate)
hoist0 = problem.add_object("hoist0", Hoist)
hoist1 = problem.add_object("hoist1", Hoist)
hoist2 = problem.add_object("hoist2", Hoist)

# 2. Set Initial State
problem.set_initial_value(at(pallet0, depot0), True)
problem.set_initial_value(clear(crate1), True)
problem.set_initial_value(at(pallet1, distributor0), True)
problem.set_initial_value(clear(crate0), True)
problem.set_initial_value(at(pallet2, distributor1), True)
problem.set_initial_value(clear(pallet2), True)
problem.set_initial_value(at(truck0, distributor1), True)
problem.set_initial_value(at(truck1, depot0), True)
problem.set_initial_value(at(hoist0, depot0), True)
problem.set_initial_value(available(hoist0), True)
problem.set_initial_value(at(hoist1, distributor0), True)
problem.set_initial_value(available(hoist1), True)
problem.set_initial_value(at(hoist2, distributor1), True)
problem.set_initial_value(available(hoist2), True)
problem.set_initial_value(at(crate0, distributor0), True)
problem.set_initial_value(on(crate0, pallet1), True)
problem.set_initial_value(at(crate1, depot0), True)
problem.set_initial_value(on(crate1, pallet0), True)

# 3. Set Goal Network
gn = PartialOrderGoalNetwork()
gn.add(at(pallet0, depot0), at(pallet0, depot0), on(crate1, pallet1), on(crate0, pallet2))
problem.set_goal_network(gn)


In [7]:
from phgn_planner.factored_uct import PHGNPlanner

cfg = UCTConfig(
    n_rollouts = 1000,  # number of rollouts to perform
    horizon = 20,  # maximum depth of each rollout
    budget = 100,  # the maximum cost budget for a single run
    exploration_const = 2**0.5,  # exploration-exploitation tradeoff (c) value for UCB
    normalize_exploration_const = True,  # whether to normalize c in UCB
    n_init = 5,  # initial visit count (delta)
    risk_factor = -0.1,  # risk factor (lambda) for GUBS criterion
    goal_utility = 1,  # goal utility constant for GUBS criterion
    h_util = lambda _: 1,  # utility heuristic
    h_ptg = lambda _: 1,  # probability-to-goal heuristic
    seed = 2,  # random seed
    show_progress = True,  # whether to print planning progress to stdout
)


from phgn_planner.experiments.domains.transport import transport

from phgn_planner.experiments.domains.depot import depot

from phgn_planner.experiments.domains.childsnack import childsnack

from phgn_planner.experiments.domains.satellite import satellite


problem = satellite(problem_instance=8)

planner = PHGNPlanner(cfg)

planner.run(problem, show_progress=True)

PartialOrderGoalNetwork:
Subgoals:
have_image(Phenomenon4, thermograph)
have_image(Star5, x_ray)
have_image(Phenomenon6, x_ray)

Orderings:
have_image(Phenomenon4, thermograph) -> have_image(Star5, x_ray)
have_image(Star5, x_ray) -> have_image(Phenomenon6, x_ray)

Selected method ('method0', (Star5, satellite0, Phenomenon4, instrument01, thermograph))
PartialOrderGoalNetwork:
Subgoals:
have_image(Phenomenon4, thermograph)
have_image(Star5, x_ray)
have_image(Phenomenon6, x_ray)
power_on(instrument01)
pointing(satellite0, Phenomenon4)
have_image(Phenomenon4, thermograph)

Orderings:
have_image(Phenomenon4, thermograph) -> have_image(Star5, x_ray)
have_image(Star5, x_ray) -> have_image(Phenomenon6, x_ray)
power_on(instrument01) -> pointing(satellite0, Phenomenon4)
pointing(satellite0, Phenomenon4) -> have_image(Phenomenon4, thermograph)
have_image(Phenomenon4, thermograph) -> have_image(Phenomenon4, thermograph)

	Selected action ('switch_on', (instrument01, satellite0))
PartialOrderGoalN

(PlanningResult.SUCCESS, 28, 13281)

In [5]:
problem

problem name = satellite2

types = [Object, Instrument - Object, Satellite - Object, Mode - Object, Direction - Object, ImageDirection - Direction, CalibDirection - Direction]

fluents = [
  bool on_board[i=Instrument - Object, s=Satellite - Object]
  bool supports[i=Instrument - Object, m=Mode - Object]
  bool pointing[s=Satellite - Object, d=Direction - Object]
  bool power_avail[s=Satellite - Object]
  bool power_on[i=Instrument - Object]
  bool calibrated[i=Instrument - Object]
  bool have_image[d=ImageDirection - Direction, m=Mode - Object]
  bool calibration_target[i=Instrument - Object, cd=CalibDirection - Direction]
  bool is_moving[d=ImageDirection - Direction]
  bool is_stationary[d=ImageDirection - Direction]
  bool resolved_motion[d=ImageDirection - Direction]
]

actions = [
  action turn_to(Satellite - Object t_s, Direction - Object t_d_new, Direction - Object t_d_prev) {
    preconditions = [
      pointing(t_s, t_d_prev)
    ]
    effects = [
      pointing(t_s, t_d_new)

In [6]:
pro

NameError: name 'pro' is not defined

In [ ]:
simulator = PHGNSimulator(problem)
s0 = simulator.get_initial_state()
print(s0)

{at(tray1, kitchen): true, at(tray2, kitchen): true, at(tray3, kitchen): true, at_kitchen_bread(bread1): true, at_kitchen_bread(bread2): true, at_kitchen_bread(bread3): true, at_kitchen_content(content1): true, at_kitchen_content(content2): true, at_kitchen_content(content3): true, at_kitchen_content(content4): true, at_kitchen_content(content5): true, at_kitchen_content(content6): true, at_kitchen_content(content7): true, at_kitchen_content(content8): true, at_kitchen_content(content9): true, at_kitchen_content(content10): true, no_gluten_bread(bread2): true, no_gluten_content(content2): true, no_gluten_content(content8): true, no_gluten_content(content4): true, no_gluten_content(content1): true, allergic_gluten(child1): true, not_allergic_gluten(child2): true, not_allergic_gluten(child3): true, waiting(child1, table2): true, waiting(child2, table1): true, waiting(child3, table1): true, notexist(sandw1): true, notexist(sandw2): true, notexist(sandw3): true, at_kitchen_sandwich(sandw1)

In [ ]:
gtn = problem.goal_network

unconstrained = gtn.get_unconstrained().copy()
while unconstrained:
    subgoal = unconstrained.pop()
    print(subgoal)
    if simulator.satisfies(s0, [subgoal.get_content()]):
        for successor in gtn.network.successors(subgoal):
            unconstrained.add(successor)
        gtn.release(subgoal)
        print("removed")

In [ ]:
x = list(gtn.network.nodes())[0]
gtn.release(x)

IndexError: list index out of range

In [ ]:
print(gtn)
print(x)

PartialOrderGoalNetwork:
Subgoals:
served(child2)
served(child3)
served(child1)
served(child1)
served(child1)
ontray(sandw2, tray1)
at(tray1, table2)
served(child1)

Orderings:
served(child2) -> served(child3)
served(child1) -> served(child1)
served(child1) -> served(child1)
ontray(sandw2, tray1) -> at(tray1, table2)
at(tray1, table2) -> served(child1)
served(child1) -> served(child1)

served(child1)


In [ ]:
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
served(child2)
served(child3)
served(child1)
served(child1)
served(child1)
ontray(sandw2, tray1)
at(tray1, table2)
served(child1)

Orderings:
served(child2) -> served(child3)
served(child1) -> served(child1)
served(child1) -> served(child1)
ontray(sandw2, tray1) -> at(tray1, table2)
at(tray1, table2) -> served(child1)
served(child1) -> served(child1)



In [ ]:
from phgn_planner.experiments.domains.transport import transport

problem = transport(problem_instance=1)

In [ ]:
# --- TRANSPORT DOMAIN DEFINITION (from previous response) ---
problem = PHGNProblem()

# 1. Define UserTypes (with hierarchy)
Object = UserType("Object")
Locatable = UserType("Locatable", father=Object)
Package = UserType("Package", father=Locatable)
Vehicle = UserType("Vehicle", father=Locatable)
CapacityNumber = UserType("CapacityNumber", father=Object)
Location = UserType("Location", father=Object)
Target = UserType("Target", father=Object)

# 2. Define Fluents (Predicates)
road = problem.add_fluent(
    "road", BoolType(), default_initial_value=False, loc1=Location, loc2=Location
)
at = problem.add_fluent(
    "at", BoolType(), default_initial_value=False, obj=Locatable, loc=Location
)
in_vehicle = problem.add_fluent(
    "in_vehicle", BoolType(), default_initial_value=False, p=Package, v=Vehicle
)
current_capacity_level = problem.add_fluent(
    "current_capacity_level",
    BoolType(),
    default_initial_value=False,
    v=Vehicle,
    s=CapacityNumber,
)
capacity_predecessor = problem.add_fluent(
    "capacity_predecessor",
    BoolType(),
    default_initial_value=False,
    s1=CapacityNumber,
    s2=CapacityNumber,
)

# 3. Define Actions
drive = InstantaneousAction("drive", v=Vehicle, l1=Location, l2=Location)
drive.add_precondition(at(drive.v, drive.l1))
drive.add_precondition(road(drive.l1, drive.l2))
drive.add_effect(at(drive.v, drive.l1), False)
drive.add_effect(at(drive.v, drive.l2), True)
problem.add_action(drive)

pick_up = InstantaneousAction(
    "pick_up",
    v=Vehicle,
    l=Location,
    p=Package,
    s1=CapacityNumber,
    s2=CapacityNumber,
)
pick_up.add_precondition(at(pick_up.v, pick_up.l))
pick_up.add_precondition(at(pick_up.p, pick_up.l))
pick_up.add_precondition(capacity_predecessor(pick_up.s1, pick_up.s2))
pick_up.add_precondition(current_capacity_level(pick_up.v, pick_up.s2))
pick_up.add_effect(at(pick_up.p, pick_up.l), False)
pick_up.add_effect(in_vehicle(pick_up.p, pick_up.v), True)
pick_up.add_effect(current_capacity_level(pick_up.v, pick_up.s1), True)
pick_up.add_effect(current_capacity_level(pick_up.v, pick_up.s2), False)
problem.add_action(pick_up)

drop = ProbabilisticAction(
    "drop", v=Vehicle, l=Location, p=Package, s1=CapacityNumber, s2=CapacityNumber
)
drop.add_precondition(at(drop.v, drop.l))
drop.add_precondition(in_vehicle(drop.p, drop.v))
drop.add_precondition(capacity_predecessor(drop.s1, drop.s2))
drop.add_precondition(current_capacity_level(drop.v, drop.s1))

drop.add_outcome("success", 0.5)
drop.add_effect("success", in_vehicle(drop.p, drop.v), False)
drop.add_effect("success", at(drop.p, drop.l), True)
drop.add_effect("success", current_capacity_level(drop.v, drop.s2), True)
drop.add_effect("success", current_capacity_level(drop.v, drop.s1), False)
drop.add_outcome("failure", 0.5)
problem.add_action(drop)

deliver = PHGNMethod(
    "deliver",
    package=Package,
    vehicle=Vehicle,
    source=Location,
    destination=Location,
)
deliver.add_precondition(at(deliver.package, deliver.source))
deliver.add_precondition(Not(Equals(deliver.source, deliver.destination)))
gn = PartialOrderGoalNetwork()
gn.add(
    at(deliver.vehicle, deliver.source),
    in_vehicle(deliver.package, deliver.vehicle),
    And(
        at(deliver.vehicle, deliver.destination),
        in_vehicle(deliver.package, deliver.vehicle),
    ),
    at(deliver.package, deliver.destination),
)
deliver.set_goal_network(gn)
problem.add_method(deliver)

# 1. Define Objects
package_0 = problem.add_object("package_0", Package)
package_1 = problem.add_object("package_1", Package)

capacity_0 = problem.add_object("capacity_0", CapacityNumber)
capacity_1 = problem.add_object("capacity_1", CapacityNumber)
capacity_2 = problem.add_object("capacity_2", CapacityNumber)

city_loc_0 = problem.add_object("city_loc_0", Location)
city_loc_1 = problem.add_object("city_loc_1", Location)
city_loc_2 = problem.add_object("city_loc_2", Location)

truck_0 = problem.add_object("truck_0", Vehicle)

# 2. Set Initial State
problem.set_initial_value(capacity_predecessor(capacity_0, capacity_1), True)
problem.set_initial_value(capacity_predecessor(capacity_1, capacity_2), True)

problem.set_initial_value(road(city_loc_0, city_loc_1), True)
problem.set_initial_value(road(city_loc_1, city_loc_0), True)
problem.set_initial_value(road(city_loc_1, city_loc_2), True)
problem.set_initial_value(road(city_loc_2, city_loc_1), True)

problem.set_initial_value(at(package_0, city_loc_1), True)
problem.set_initial_value(at(package_1, city_loc_1), True)
problem.set_initial_value(at(truck_0, city_loc_2), True)

problem.set_initial_value(current_capacity_level(truck_0, capacity_2), True)

gn = PartialOrderGoalNetwork()
gn.add(at(package_0, city_loc_0), at(package_1, city_loc_2))
problem.set_goal_network(gn)
problem.add_goal(at(package_0, city_loc_0))

In [ ]:
gn = problem.goal_network
print(gn)

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)



In [ ]:
gn2 = gn.copy()

In [ ]:
print(gn2)


PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)



In [ ]:
node = list(gn.network.nodes())[0]
print(node)

gn3 = gn.copy().release(node)

at(package_0, city_loc_0)


In [ ]:
print(gn3)
print(gn2)
print(gn)

PartialOrderGoalNetwork:
Subgoals:
at(package_1, city_loc_2)

Orderings:

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)



In [ ]:
gn2.release(node)
print(hash(gn3) == hash(gn2))

True


In [ ]:
print(hash(gn3) == hash(gn2))

True


In [ ]:
from phgn_planner.unfactored_uct import PHGNPlanner

cfg = UCTConfig(
    n_rollouts = 2000,  # number of rollouts to perform
    horizon = 20,  # maximum depth of each rollout
    budget = 200,  # the maximum cost budget for a single run
    exploration_const = 2**0.5,  # exploration-exploitation tradeoff (c) value for UCB
    normalize_exploration_const = True,  # whether to normalize c in UCB
    n_init = 5,  # initial visit count (delta)
    risk_factor = -0.1,  # risk factor (lambda) for GUBS criterion
    goal_utility = 1,  # goal utility constant for GUBS criterion
    h_util = lambda _: 1,  # utility heuristic
    h_ptg = lambda _: 1,  # probability-to-goal heuristic
    seed = None,  # random seed
    show_progress = False,  # whether to print planning progress to stdout
)

planner = PHGNPlanner(cfg)

planner.run(problem, show_progress=True)

KeyboardInterrupt: 

In [ ]:
from phgn_planner.unfactored_tree import TreeNodeFactory
from phgn_planner.unfactored_tree import TreePolicy, UCBPolicy

simulator = PHGNSimulator(problem)
factory = TreeNodeFactory(simulator)
n0 = factory.new_node(simulator.get_initial_state(), problem.goal_network)
grounder = PHGNGrounderHelper(problem)
print(n0)
print(problem.goal_network.get_unconstrained())

NameError: name 'PHGNGrounderHelper' is not defined

In [ ]:
ucb_policy=UCBPolicy(simulator)
# m = n0.get_applicable_methods().pop()
m = n0.select(ucb_policy)
gtn = problem.goal_network
print(m[0].name, m[1])

drive (truck_0, city_loc_2, city_loc_1)


In [ ]:
m = list(n0.get_applicable_methods())[0]

In [ ]:
simulator.is_relevant(*m, problem.goal_network)

set()

In [ ]:
print(problem.goal_network._unconstrained_cache)

{at(package_0, city_loc_0)}


In [ ]:
g_method = grounder.ground_method(m[0], m[1])
print(g_method.postconditions)
problem.goal_network.get_unconstrained()

[at(package_1, city_loc_2)]


{at(package_0, city_loc_0)}

In [ ]:
simulator.is_relevant(m[0], m[1], problem.goal_network)

set()

In [ ]:
n0.get_applicable_methods()
ucb_policy=UCBPolicy(simulator)
selected = n0.select(ucb_policy)
print(selected[0].name, selected[1])

drive (truck_0, city_loc_2, city_loc_1)


In [ ]:
s0 = simulator.get_initial_state()

In [ ]:
print(s0 in factory._nodes)
factory._nodes[s0]

True


{<unified_planning.model.phgn.goal_network.PartialOrderGoalNetwork at 0x75dd69ba4210>: <phgn_planner.unfactored_tree.TreeNode at 0x75dd69b57d80>}

In [ ]:
factory._nodes[s0]

{<unified_planning.model.phgn.goal_network.PartialOrderGoalNetwork at 0x75dd69ba4210>: <phgn_planner.unfactored_tree.TreeNode at 0x75dd69b57d80>}

In [ ]:
gn = PartialOrderGoalNetwork()
gn.add(loc(box).Equals(l4))

(loc(box) == l4)

In [ ]:
gtn == list(factory._nodes[s0].keys())[0]

True

In [ ]:
g2 = gtn.copy()
g2.decompose(g_method.goal_network)
g2 == gtn

False

In [ ]:
print(g2)
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)
at(truck_0, city_loc_1)
in_vehicle(package_1, truck_0)
(at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0))
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)
at(truck_0, city_loc_1) -> in_vehicle(package_1, truck_0)
in_vehicle(package_1, truck_0) -> (at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0))
(at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0)) -> at(package_1, city_loc_2)

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)



In [ ]:
print(g2)
print(g3)
print(hash(g2))
print(hash(g3))
print(g2.network == g3.network)
print(g2 == g3)

PartialOrderGoalNetwork:
Subgoals:
at(package_0, city_loc_0)
at(package_1, city_loc_2)
at(truck_0, city_loc_1)
in_vehicle(package_1, truck_0)
(at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0))
at(package_1, city_loc_2)

Orderings:
at(package_0, city_loc_0) -> at(package_1, city_loc_2)
at(truck_0, city_loc_1) -> in_vehicle(package_1, truck_0)
in_vehicle(package_1, truck_0) -> (at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0))
(at(truck_0, city_loc_2) and in_vehicle(package_1, truck_0)) -> at(package_1, city_loc_2)



NameError: name 'g3' is not defined

In [ ]:
g3 = gtn.copy()
g3.decompose(g_method.goal_network)
print(g3)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)



In [ ]:
len(factory._nodes.keys())

1

In [ ]:
n2 = factory.new_node(s0, g2)

In [ ]:
print(n2)

Goal Network: PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)

Expanded: False
Applicable Actions/Methods:
Visits: 0



In [ ]:
g2 == g3

False

In [ ]:
print(problem.goal_network)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)

Orderings:



In [ ]:
a = list(n0.get_applicable_actions())[0]
s1 = simulator.apply(s0, *a)
factory.new_node(s1, problem.goal_network)

In [ ]:
gtn.copy().decompose()

TypeError: PartialOrderGoalNetwork.decompose() missing 1 required positional argument: 'gtn'

In [ ]:
import networkx as nx

graph = nx.DiGraph([(1, 2), (2, 3)])
hash(graph)

7951715168431

In [ ]:
graph.add_node(5)

In [ ]:
hash(graph.copy())

7951800848589

In [ ]:
hash(graph)

7951715168431

In [ ]:
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)



In [ ]:
gtn.decompose(g_method.goal_network)

In [ ]:
hash(gtn)

356005156373860591

In [ ]:
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)

Orderings:



In [ ]:
print(gtn)
goal_node = list(gtn.network.nodes())[0]
gtn.release(goal_node)
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)



In [ ]:
g_method = grounder.ground_method(m[0], m[1])
g_method.goal_network
gtn.decompose(g_method.goal_network, m[2])
print(gtn)

PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)
(loc(box) == l4) -> (loc(box) == l4)



In [ ]:
list(gtn.network.nodes())[0] is list(gtn.copy().network.nodes())[0]

True

In [ ]:
print(n0)

Goal Network: PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)
(loc(box) == on_robot)
((loc(box) == on_robot) and (loc(robot) == l4))
(loc(box) == l4)

Orderings:
(loc(box) == on_robot) -> ((loc(box) == on_robot) and (loc(robot) == l4))
((loc(box) == on_robot) and (loc(robot) == l4)) -> (loc(box) == l4)
(loc(box) == l4) -> (loc(box) == l4)

Expanded: False
Applicable Actions/Methods:
  ('move_without_key', (l3, l6)): Q=0.0, N=0.0
  ('move_without_key', (l3, l2)): Q=0.0, N=0.0
  ('move_box', (l1,)): Q=0.0, N=0.0
  ('move_box', (l2,)): Q=0.0, N=0.0
  ('move_box', (l3,)): Q=0.0, N=0.0
  ('move_box', (l4,)): Q=0.0, N=0.0
  ('move_box', (l5,)): Q=0.0, N=0.0
  ('move_box', (l6,)): Q=0.0, N=0.0
  ('get_box', (l5,)): Q=0.0, N=0.0
  ('get_key', (l6,)): Q=0.0, N=0.0
Visits: 0



In [ ]:
simulator = PHGNSimulator(problem)

In [ ]:
s0 = simulator.get_initial_state()

In [ ]:
from phgn_planner.unfactored_tree import TreeNode, TreeNodeFactory

factory = TreeNodeFactory(simulator)

node0 = factory.new_node(s0, problem.goal_network)

print(node0)
print(factory._nodes)

Goal Network: PartialOrderGoalNetwork:
Subgoals:
(loc(box) == l4)

Orderings:

Expanded: False
Applicable Actions/Methods:
Visits: 0

{{adjacent(l1, l2): true, adjacent(l1, l4): true, adjacent(l2, l1): true, adjacent(l2, l5): true, adjacent(l2, l3): true, adjacent(l3, l2): true, adjacent(l3, l6): true, adjacent(l4, l1): true, adjacent(l5, l2): true, adjacent(l6, l3): true, locked(l2, l5): true, loc(robot): l3, loc(key): l6, loc(box): l5, adjacent(l1, l1): false, adjacent(l3, l1): false, adjacent(l5, l1): false, adjacent(l6, l1): false, adjacent(l2, l2): false, adjacent(l4, l2): false, adjacent(l6, l2): false, adjacent(l1, l3): false, adjacent(l3, l3): false, adjacent(l4, l3): false, adjacent(l5, l3): false, adjacent(l2, l4): false, adjacent(l3, l4): false, adjacent(l4, l4): false, adjacent(l5, l4): false, adjacent(l6, l4): false, adjacent(l1, l5): false, adjacent(l3, l5): false, adjacent(l4, l5): false, adjacent(l5, l5): false, adjacent(l6, l5): false, adjacent(l1, l6): false, adjacent

In [ ]:
gn = problem.goal_network
gn2 = gn.copy()
gn.get_unconstrained() is gn2.get_unconstrained()

False

In [ ]:
a = list(node0.get_applicable_actions())[0]
print(a[0].name, a[1])

move_without_key (l3, l6)


In [ ]:
s1 = simulator.apply(s0, *a)
print(s1 == s0)

False


In [ ]:
node1 = factory.new_node(s1, problem.goal_network)

In [ ]:
node1 is node0

False

In [ ]:
factory._nodes[s1][problem.goal_network]

In [ ]:
hash(s1) == hash(s0)

False

In [ ]:
s1 == s0

False

In [ ]:
len(factory._nodes)

2

In [ ]:
d = {}
d[(s0, )] = 1
d[(1, 2)] = 3

In [ ]:
d = {}
d[(1, 2)] = 'a'

In [ ]:
(1, 2) in d

True

In [ ]:
d.keys()

dict_keys([(1, 2)])

In [ ]:
d[(s0, )] = 1

print((s0, ) in d)
print(d.get((s0, ), 0))
print((1, 2) in d)

True
1
True


In [ ]:
d[problem.goal_network] = 3
d.keys()
len(d)
d[problem.goal_network]
problem.goal_network in d

True

In [ ]:
for k in d.keys():
    print(hash(k))

-3550055125485641917
-5090205872558970714
8070069192255


In [ ]:
hash((s0, problem.goal_network))

-8726898654265686008

In [ ]:
factory._nodes

{{adjacent(l1, l2): true, adjacent(l1, l4): true, adjacent(l2, l1): true, adjacent(l2, l5): true, adjacent(l2, l3): true, adjacent(l3, l2): true, adjacent(l3, l6): true, adjacent(l4, l1): true, adjacent(l5, l2): true, adjacent(l6, l3): true, locked(l2, l5): true, loc(robot): l3, loc(key): l6, loc(box): l5, adjacent(l1, l1): false, adjacent(l3, l1): false, adjacent(l5, l1): false, adjacent(l6, l1): false, adjacent(l2, l2): false, adjacent(l4, l2): false, adjacent(l6, l2): false, adjacent(l1, l3): false, adjacent(l3, l3): false, adjacent(l4, l3): false, adjacent(l5, l3): false, adjacent(l2, l4): false, adjacent(l3, l4): false, adjacent(l4, l4): false, adjacent(l5, l4): false, adjacent(l6, l4): false, adjacent(l1, l5): false, adjacent(l3, l5): false, adjacent(l4, l5): false, adjacent(l5, l5): false, adjacent(l6, l5): false, adjacent(l1, l6): false, adjacent(l2, l6): false, adjacent(l4, l6): false, adjacent(l5, l6): false, adjacent(l6, l6): false, locked(l1, l1): false, locked(l2, l1): fal

In [ ]:
factory._nodes[(s0, problem.goal_network)]

KeyError: ({adjacent(l1, l2): true, adjacent(l1, l4): true, adjacent(l2, l1): true, adjacent(l2, l5): true, adjacent(l2, l3): true, adjacent(l3, l2): true, adjacent(l3, l6): true, adjacent(l4, l1): true, adjacent(l5, l2): true, adjacent(l6, l3): true, locked(l2, l5): true, loc(robot): l3, loc(key): l6, loc(box): l5, adjacent(l1, l1): false, adjacent(l3, l1): false, adjacent(l5, l1): false, adjacent(l6, l1): false, adjacent(l2, l2): false, adjacent(l4, l2): false, adjacent(l6, l2): false, adjacent(l1, l3): false, adjacent(l3, l3): false, adjacent(l4, l3): false, adjacent(l5, l3): false, adjacent(l2, l4): false, adjacent(l3, l4): false, adjacent(l4, l4): false, adjacent(l5, l4): false, adjacent(l6, l4): false, adjacent(l1, l5): false, adjacent(l3, l5): false, adjacent(l4, l5): false, adjacent(l5, l5): false, adjacent(l6, l5): false, adjacent(l1, l6): false, adjacent(l2, l6): false, adjacent(l4, l6): false, adjacent(l5, l6): false, adjacent(l6, l6): false, locked(l1, l1): false, locked(l2

In [ ]:
list(factory._nodes.keys())[0] == (s0, problem.goal_network)

True

In [ ]:
node1 = factory.new_node(s1, problem.goal_network)

In [ ]:
factory._nodes.__repr__()

'{({adjacent(l1, l2): true, adjacent(l1, l4): true, adjacent(l2, l1): true, adjacent(l2, l5): true, adjacent(l2, l3): true, adjacent(l3, l2): true, adjacent(l3, l6): true, adjacent(l4, l1): true, adjacent(l5, l2): true, adjacent(l6, l3): true, locked(l2, l5): true, loc(robot): l3, loc(key): l6, loc(box): l5, adjacent(l1, l1): false, adjacent(l3, l1): false, adjacent(l5, l1): false, adjacent(l6, l1): false, adjacent(l2, l2): false, adjacent(l4, l2): false, adjacent(l6, l2): false, adjacent(l1, l3): false, adjacent(l3, l3): false, adjacent(l4, l3): false, adjacent(l5, l3): false, adjacent(l2, l4): false, adjacent(l3, l4): false, adjacent(l4, l4): false, adjacent(l5, l4): false, adjacent(l6, l4): false, adjacent(l1, l5): false, adjacent(l3, l5): false, adjacent(l4, l5): false, adjacent(l5, l5): false, adjacent(l6, l5): false, adjacent(l1, l6): false, adjacent(l2, l6): false, adjacent(l4, l6): false, adjacent(l5, l6): false, adjacent(l6, l6): false, locked(l1, l1): false, locked(l2, l1): f

In [ ]:
nodes = list(factory._nodes.keys())
len(nodes)

2

In [ ]:
hash(nodes[1][0]) == hash(s0)

True

In [ ]:
(nodes[0][0], problem.goal_network) in factory._nodes

False

In [ ]:
print(hash(nodes[0]))
print(hash(nodes[1]))

947165744329167765
1896170402384524657


In [ ]:
from phgn_planner.unfactored_uct import PHGNPlanner

planner = PHGNPlanner()

planner.run(problem, show_progress=True, n_rollouts=5000)

KeyboardInterrupt: 

In [ ]:
u = list(node.get_applicable_methods())[0]
grounder = PHGNGrounderHelper(problem)
grounder.ground_method(*u).goal_network

gn.decompose(grounder.ground_method(*u).goal_network)

TotalOrderGoalNetwork[(loc(box) == on_robot), ((loc(box) == on_robot) and (loc(robot) == l4)), (loc(box) == l4)]

In [ ]:
gn

TotalOrderGoalNetwork[(loc(box) == on_robot), ((loc(box) == on_robot) and (loc(robot) == l4)), (loc(box) == l4)]

In [ ]:
problem.method("move_box").postconditions

[(loc(box) == to_loc)]

In [ ]:
simulator = PHGNSimulator(problem)

In [ ]:
s = simulator.get_initial_state()

In [ ]:
simulator.apply()

TypeError: SequentialSimulatorMixin.apply() missing 2 required positional arguments: 'state' and 'action_or_action_instance'

In [ ]:
problem.goal_network.get_unconstrained()

In [ ]:
gn = TotalOrderGoalNetwork()
gn.add(loc(box).Equals(l7))
gn

In [ ]:
simulator = PHGNSimulator(problem)

In [ ]:
s0 = simulator.get_initial_state()

In [ ]:
for a, p in simulator.get_applicable_actions(s0):
    print(a.name, p)
    print(type(p[0]))

for m, p in simulator.get_applicable_methods(s0):
    print(m.name, p)

In [ ]:
gn.get_unconstrained()

In [ ]:
for method, params in simulator.get_applicable_methods(s0):
    if simulator.is_relevant(method, params, gn):
        print(method.name, params)


In [ ]:
action, params = next(simulator.get_applicable_actions(s0))

In [ ]:
action, params

In [ ]:
s1 = simulator.apply(s0, action, params)
s1a = simulator.apply(s0, action, params)

In [ ]:
method, params = [(method, param) for (method, param) in simulator.get_applicable_methods(s0) if simulator.is_relevant(method, param, gn)][0]

gmethod = simulator._phgn_grounder.ground_method(method, params)

In [ ]:
gmethod.goal_network

In [ ]:
print(gn)
gn.decompose(gmethod.goal_network)
print(gn)

In [ ]:
print(hash(s1))
print(hash(s1a))
print(hash(s0))


In [ ]:
problem.method("move_box")

In [ ]:
simulator.is_relevant(problem.method("move_box"), tuple([l7]), gn)

In [ ]:
problem = key_warehouse_problem()

In [ ]:
problem.methods

In [ ]:
move_box = PHGNMethod("move_box", to_loc=GroundLocation)
move_box.add_precondition(loc(box).Equals(on_robot).Not())
move_box.add_subgoals(loc(box).Equals(on_robot), And(loc(box).Equals(on_robot), loc(robot).Equals(move_box.to_loc)), loc(box).Equals(move_box.to_loc))

        # move_box = PHGNMethod("move_box", to_loc=GroundLocation)
        # move_box.add_subgoal("get_box", loc(box).Equals(on_robot))
        # move_box.add_subgoal_precondition("get_box", loc(box).Equals(on_robot).Not())
        # move_box.add_subgoal("move_with_box", loc(box).Equals(on_robot))
        # move_box.add_subgoal("move_with_box", loc(robot).Equals(move_box.to_loc))
        # move_box.add_subgoal_precondition("move_with_box", loc(box).Equals(on_robot))
        # move_box.add_subgoal_precondition("move_with_box", loc(robot).Equals(move_box.to_loc).Not())
        # move_box.add_subgoal("put_down_box", loc(box).Equals(move_box.to_loc))
        # move_box.add_subgoal_precondition("put_down_box", loc(box).Equals(on_robot))
        # move_box.add_subgoal_precondition("put_down_box", loc(robot).Equals(move_box.to_loc))
        # move_box.add_postcondition(loc(box).Equals(move_box.to_loc))
        # problem.add_method(move_box)